# WBS Project 2 - Eniac - Cleaning and Categories

Mathis Lammert

mathislammert@gmail.com

Created: 2024-03-11

## Part 1: Cleaning

In [2]:
# Import libraries
import pandas as pd

### load data

In [3]:
folder = ".\data\\"

brands = pd.DataFrame(pd.read_csv(folder+"brands.csv"))
items = pd.DataFrame(pd.read_csv(folder+"orderlines.csv"))
orders = pd.DataFrame(pd.read_csv(folder+"orders.csv"))
products = pd.DataFrame(pd.read_csv(folder+"products.csv"))

In [ ]:
# load data online
# def gd_path(file_id):
#     """Generate a shareable link from Google Drive file id."""
#     return f"https://drive.google.com/uc?export=download&id={file_id}"

# # Google Drive file ids
# files_id = {
#     "brands": "1m1ThDDIYRTTii-rqM5SEQjJ8McidJskD",
#     "items": "1FYhN_2AzTBFuWcfHaRuKcuCE6CWXsWtG",
#     "orders": "1Vu0q91qZw6lqhIqbjoXYvYAQTmVHh6uZ",
#     "products": "1afxwDXfl-7cQ_qLwyDitfcCx3u7WMvkU"
# }

# # Read data from Google Drive
# orders = pd.DataFrame(pd.read_csv(gd_path(files_id["orders"])))
# items = pd.DataFrame(pd.read_csv(gd_path(files_id["items"])))
# products = pd.DataFrame(pd.read_csv(gd_path(files_id["products"])))
# brands = pd.DataFrame(pd.read_csv(gd_path(files_id["brands"])))

### Abnormalities

Exploration found several abnormalities:
- items.product_id is empty - can be dropped
- product_quantity has max 999 - thats not realistic and seems like a placeholder - investigate
- dates are not dtype date in orders
- items.unit_price is object but should be float - are non-digits in there?
- there is on unit_price with "-119.00"
- products.promo_price 
- correct prices in product table as well (same problem as in items?)
- products.type is an object but supposed to be an numerical code 
- promo price is way bigger than regular price in some instances
- why are promotion prices so big?
- how to calculate revenue
- possible product categories
- some prices are NaA
- some order_ids are not there in items..?!?!
- the prices actually dont always have 2 decimals - my approach is not valid
- price relationships are weird - price, unit_price and promo_price
- there are duplicates - must be checked and dropped before
- app. there are 
- maybe drop incomplete cases (shopping basket, etc)? how many cases?

### General Cleaning 

In [ ]:
##  data types "date"
orders["created_date"] = pd.to_datetime(orders["created_date"])   # as date
items["date"] = pd.to_datetime(items["date"]) # as date

In [ ]:
## drop unused column
items = items.drop(columns="product_id")

In [ ]:
# Change column names
items.rename(columns={"id_order": "order_id"}, inplace=True)   # so that orders and items have identical order_id -names
orders.rename(columns={"created_date":"date"}, inplace=True)

In [ ]:
# Check and Drop duplicates
items.duplicated().value_counts() # -> no duplicates in items
orders.duplicated().value_counts() # -> no duplicates in orders
products.duplicated().value_counts() # -> many duplicates in products -
products = products.loc[~products.duplicated()] # -> drop them
brands.duplicated().value_counts() # -> no duplicates in brands

In [ ]:
## Check for and remove missing values
products.price.isna().value_counts() # -> only 46 products dont have a price -> delete them!
products = products.loc[~products.price.isna()]

products.isna().sum() # -> other NAs are in desc - leave it, not important for now 
items.isna().sum() # -> no NAs here
orders.isna().sum() # -> 5 NAs in total_paid - need to delete it
del_orderid = orders.loc[orders.total_paid.isna(), "order_id"].tolist() # create list of ID that need to be deleted..
orders = orders.loc[~orders.order_id.isin(del_orderid)] # .. in orders
items = items.loc[~items.order_id.isin(del_orderid)]  #  .. in items

### Clean Prices

In [ ]:
## prices - whats the problem?
items.loc[items.unit_price.astype("str").str.contains("[^0-9.]")]  # search for substrings that are not digits or points -> there is one value with "-119.00"
#items.unit_price.astype("float") # -> apper. there is at least on case where the 
items.loc[items.unit_price.str.len() > 6] # -> this seems to  systematic: thousands and decimals are seperated by point

Build a Regex that finds the faulty price patterns

In [ ]:
regexfilter = r'(\..*?\..*)|(\.\d{3,}$)'
products.price.str.contains(regexfilter).value_counts(normalize=True) # this finds these weird patterns ("10.939.496") -> they account for 5.1% of the data
products.loc[products.price.str.contains(regexfilter)].sample(30)
products = products.loc[~products.price.str.contains(regexfilter)] # until a better solution is found - delete them

Use a function to correct the price, based on this regex pattern

In [ ]:
def price_cleaner(col):
    # Use regex to replace points that are followed by three digits and then by another point or end of the string
    col = col.str.replace(r'\.(\d{3})(?=\.|$)', r'\1', regex=True)

    # Convert to float
    col = col.astype(float)
    return col

items["unit_price"] = price_cleaner(items["unit_price"])
products["price"] = price_cleaner(products["price"])
#products["promo_price"] = price_cleaner(products["promo_price"])
products.sort_values("price", ascending=False).head(30)

some price have negative values

In [ ]:
## find and delete orders having negative prices in Items or order 
items.loc[items.unit_price.astype("str").str.contains("[^0-9.]")]  # -> order id 365886
items.loc[items.order_id == 365886] 
orders.loc[orders.order_id == 365886]
order = orders.loc[orders.order_id != 365886]
items = items.loc[items.order_id != 365886] 

### Repair Promo_price

those patterns are even more corrupted. it seems that the decimals are at the wrong place.

Approach: Use the first digits. Extract the number of digits from "unit_price" which should have the same dimension. 

Problem: If the promo_price has one dimension less (e.g., unit price 1099€ nad promo_price 999€) this does not work 
Solution: Multiply by ten in those cases

In [ ]:
# Remove dots from 'promo_price' and ensure it's treated as a string for manipulation
products['cleaned_promo_price'] = products.promo_price.str.replace(".", "", regex=False)

# Calculate the length of the integer part of 'price'
products["price_digits"] = products.price.round(2).astype(str).str.split(".").str[0].str.len()

# Extract the first n digits from 'cleaned_promo_price', where n is determined by 'price_digits'
# And then insert a decimal point to match the 'price' format
def adjust_promo(row):
    n = row['price_digits']
    # Extract first n digits
    first_n_digits = row['cleaned_promo_price'][:n]
    last_n_digits = row['cleaned_promo_price'][n+1:n+3]
    # Insert decimal point before the last two digits
    adjusted_promo_price = first_n_digits + '.' + last_n_digits
    adjusted_promo_price = pd.Series(adjusted_promo_price).astype("float")
    return adjusted_promo_price

products['promo_price_est'] = products.apply(adjust_promo, axis=1)
products.drop(columns=['cleaned_promo_price', 'price_digits'], inplace=True) # Drop temporary columns

# some weird cases left?
(products['promo_price_est'] > products["price"]).value_counts()
products.loc[(products['promo_price_est'] > products["price"])] # it seems that in those cases (958), the above approach failed because the promo was one digit cheaper  
products.loc[(products['promo_price_est'] > products["price"]), "promo_price_est"] = (products['promo_price_est']/10).round(2) #try to just divide it by ten in those cases

# drop original promo
products.drop(columns=['promo_price'], inplace=True)

In [ ]:
# compute discount from this adjusted 
products["discount_est"] = 100-(products["promo_price_est"] * 100 / products["price"]).round(2)

product quantity is weirdly high in some cases

In [ ]:
items.sort_values("product_quantity", ascending=False).head(30)
orders[orders.order_id == 358747]   # -> app. those high numbers add up but were not actually paid but only in the shopping cart

### Consistency tests
some order_ids are not in items and vice versa.

In [ ]:
orders.loc[~orders.order_id.isin(items.order_id)] # there are 22213 entries in orders, that have no corresponding order_id in items.
orders = orders.loc[orders.order_id.isin(items.order_id)] # delete them

items.loc[~items.order_id.isin(orders.order_id)] # there are 234 entries in items, that have no corresponding order_id in orders
items = items.loc[items.order_id.isin(orders.order_id)] # delete them

orders.order_id.drop_duplicates().count() == items.order_id.drop_duplicates().count()  # nice.

In [ ]:
## SKU 
# are there duplicates?
products.sku.duplicated().value_counts() # no

In [ ]:
# check for minor cases in sku
products.loc[products.sku.str.contains("[a-z]+")] # check if there are minor cases in products.sku
items.loc[items.sku.str.contains("[a-z]+")] # or items.sku - seems to be only one relevant case (par0072)
brands.loc[brands.short == "PAR"] # for the brands relation it must be mayor case
items.loc[items.sku == "par0072", "sku"] = "PAR0072" # change it in both tables
products.loc[products.sku == "par0072", "sku"] = "PAR0072"

In [ ]:
# SKU length problems
products.sku.str.len().value_counts() # oh no, there are SKUs, that have >7 characters (1289 cases)
products.loc[products.sku.str.len() > 7].sort_values("sku") #seems like the ast characters are too much
products["sku_new"] = products.sku.str[:7] # remove the characters behind the 7th
products.sku_new.duplicated().sum() # oh no, we produced duplicates (853), but only in the sku column, the others colum seem to be unique
products.loc[products.sku_new.duplicated(keep=False)].sort_values("sku_new").head(30) 
# those are really different products. the ones with the longer sku have a text starting with "open" or "(open)" <- what does that mean?
# I guess its saver to remove those duplicates, as they originally have another key.
items.sku.str.len().value_counts() # yet, those same sku's appear in items. so probably just leave the sku be, even if its to long
products.drop(columns=['sku_new'], inplace=True)

In [ ]:
# are there SKUs in items, that are not listed in products?
(items.sku.isin(products.sku)).value_counts(normalize=True) #yes, 8742 = 2.98%
items[~items.sku.isin(products.sku)] # is SKU syntax strange? not really.. so drop it
SKU_non_corresponding_ids = items.loc[~items.sku.isin(products.sku), "order_id"]  # list of all order_ids where SKU does not correspond
orders = orders.loc[~orders.order_id.isin(SKU_non_corresponding_ids)] # remove those IDs in orders
items = items.loc[~items.order_id.isin(SKU_non_corresponding_ids)] # ..and in items, as well

### remove non-completed cases

In [ ]:
orders.value_counts("state", normalize=True) # complete cases account for 22.7% (46560 orders) - but after the cleaning we did already
ids_complete_cases = orders.loc[orders.state == "Completed", "order_id"]
orders = orders.loc[orders.order_id.isin(ids_complete_cases)] # only keep orders having those IDs 
items = items.loc[items.order_id.isin(ids_complete_cases)] # ..and items as well

### Relationship between prices

In [ ]:
# add unit_prices in items. Unit prices = quantity x unit_price
items = items.assign(unit_prices = lambda x: x.unit_price * x.product_quantity)

In [ ]:
# merge order + items + product, assign new columns for price * qty (for each price)
orders_merg = (orders
.merge(items, "left", on="order_id")
.merge(products, "left", on="sku")
.assign(unit_prices = lambda x: x.unit_price * x.product_quantity,
        prices = lambda x: x.price * x.product_quantity,
        promo_prices_est = lambda x: x.promo_price_est * x.product_quantity)
.loc[:,["order_id", "state" ,"total_paid", "product_quantity", "unit_price", "unit_prices", "price", "prices", "promo_price_est", "promo_prices_est","discount_est", "sku", "name"]]
.sort_values(["order_id"], ascending=False)
#.tail(50)
)

In [ ]:
# merge tables, group by order and aggregat to calculate the different prices (total_paid, unit_price, price, promo_price) per order. -> derive discounts, voucher and shipping costs from that
orders_agg = (orders
.merge(items, "left", on="order_id")
.merge(products, "left", on="sku")
.assign(unit_prices = lambda x: x.unit_price * x.product_quantity,
        prices = lambda x: x.price * x.product_quantity,
        promo_prices_est = lambda x: x.promo_price_est * x.product_quantity)
.groupby("order_id")
.agg({"total_paid":"mean", "unit_prices": "sum", "prices":"sum", "promo_prices_est":"sum", "product_quantity" :"sum"})
.rename(columns={"unit_prices":"sum_prices_as_shopped","prices":"sum_prices_as_listed", "promo_prices_est":"sum_dicount_prices_as_listed","product_quantity":"qty"})
.reset_index()
)

In [ ]:
# compute discount
orders_agg["discount_perc"] = (100 - (orders_agg.sum_prices_as_shopped / orders_agg.sum_prices_as_listed * 100)).round(1)
orders_agg["discount"] = (orders_agg.sum_prices_as_listed - orders_agg.sum_prices_as_shopped)
# overall it makes sense, but in some cases its negative. maybe prices changed over time?

In [ ]:
# compute discount from the estimated discount (original promo_price)
orders_agg["discount_est_perc"] = (100 - (orders_agg.sum_dicount_prices_as_listed / orders_agg.sum_prices_as_listed * 100)).round(1) 
orders_agg["discount_est"] = (orders_agg.sum_dicount_prices_as_listed - orders_agg.sum_prices_as_shopped)

In [ ]:
# compute freight costs: freight = total_paid - summed_item_prices_per_order
orders_agg["freight"] = (orders_agg["total_paid"] - orders_agg["sum_prices_as_shopped"]).round(2)
# overall reasonalbe, but some weird outliers - negative and high positive costs. 
# negative values could be vouchers/discounts (less paid then items costs) but what about the high positive values?

In [ ]:
# add date to orders_agg
orders_agg = orders_agg.merge(orders[["order_id","date"]], how="left", on="order_id")

orders_agg.isna().any() # good.

In [ ]:
# Relationship of discounts and prices:
temp_merg = products.merge(items, "right", on="sku")
temp_merg_agg = temp_merg.groupby("sku").agg({"price": ["min","max","mean"], "unit_price": ["min","max","mean"]}) 
temp_merg_agg.sample(20)  # product prices (ffrom products) represent "fixed" prices. Information about changing product prices is missing. 
(temp_merg_agg[("unit_price","mean")] != temp_merg_agg[("unit_price","max")]).value_counts()  # discount changes with different unit_prices over time

In [ ]:
# Add discount per item
temp_merg["unit_discount"] = temp_merg.price - temp_merg.unit_price
temp_merg["unit_discount_perc"] = (temp_merg["unit_discount"] / temp_merg.price *100).round(2)
items = items.merge(temp_merg[["unit_discount", "unit_discount_perc", "id"]],"left", "id")

### Consistency

In [ ]:
# investigate weird freight and discount numbers - failure of prior data cleaning or faulty data or systemic behavior in data?
orders_agg.freight.describe() 
orders_agg.nlargest(20, "freight") # the first 4 lines seem faulty - unit_price is unreasonanly low while the other prices relate to each other. drop those.
orders_agg.nsmallest(20, "freight") # the negative values might be vouchers - but there is no way to differiante between voucher and freight costs

In [ ]:
# test the other prices
orders_agg["sum_prices_as_shopped"].plot(kind="box");
#plt.show() # there are many outliers, but quiet reasonable
orders_agg["sum_prices_as_shopped"].describe()
orders_agg.nlargest(20, "sum_prices_as_shopped") # there are high prices but they are reasonable

In [ ]:
orders_agg["discount_perc"].describe()
orders_agg["discount_perc"].plot(kind="box");
#plt.show()
orders_agg.nsmallest(20, "discount_perc") # there are high prices but they are reasonable

In [ ]:
# some discounts seem not reliable
items.loc[items.unit_discount_perc < -100]

In [ ]:
# Detail look at chosen weird numbers
weird_ids = [297148,293308,512894,508825,1233154,379925,388586,347233]
orders_merg.loc[orders_merg["order_id"].isin(weird_ids)]# .sort_values("order_id")

In [ ]:
# remove weird rows in all relevant tables
items = items[~items.order_id.isin(weird_ids)]
orders = orders[~orders.order_id.isin(weird_ids)]
orders_agg = orders_agg[~orders_agg.order_id.isin(weird_ids)]
orders_merg = orders_merg[~orders_merg.order_id.isin(weird_ids)]

## Part 2: Categories

In [ ]:
# import libraries
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mp
import matplotlib.dates as mdates
import numpy as np
import addcopyfighandler
from collections import Counter
import os
import pandasai as pai
from pandasai.llm import OpenAI
import seaborn as sns
from scipy.stats import pearsonr

Clean Products table

In [ ]:
products["category"] = ""
products_all = products.copy()
products = products.merge(brands, "left", left_on=products.sku.str[0:3], right_on="short").rename(columns={"long":"brand"}).drop(columns=["short","short"])
products = products.loc[products.sku.isin(products.merge(items, "inner", "sku")["sku"].drop_duplicates())]   # CAUTION - remove unneeded products

check the pattern of the product "type"

In [ ]:
products.loc[products.category == "",["name","desc","price","type"]].value_counts("type").nlargest(20)
products.loc[products.type.isna(),["name","desc","price","type"]]

Create a function to 1. manually check common category of each type and 2. count the most frequent word in he names of those products. 

In [ ]:
def cat_info(x):
    print(products.loc[products.type == x,["name","desc","price","type","sku","brand"]])

    print(pd.DataFrame(Counter(" ".join(products.loc[products.type == x,"name"]
                            .replace(r'[^\w\s]|_', '', regex=True)
                            .replace(r' +', ' ',regex=True))
                            .split())
                .most_common(15))
    )
    print(products.loc[products.type == x,"type"].count())

Step 1: use this function to manually check each "type" in the order of its size and assign a meaninfgul name to it.
Step 2: Decide for categories and reassign all types into those categories

In [ ]:
# assign product categories
products.loc[products.type == "11865403","category"] = "Accessory"          # casing
products.loc[products.type == "12175397","category"] = "Network/Memory"     # "NAS Server"
products.loc[products.type == "1298","category"] = ""                       # Mix :/
products.loc[products.type == "11935397","category"] = "Network/Memory"     # "USB Hard Drive"
products.loc[products.type == "11905404","category"] = ""                   # Mix :/
products.loc[products.type == "1282","category"] = "Computer/Laptop"        # Mac
products.loc[products.type == "12635403","category"] = "Accessory"          # Caseing    
products.loc[products.type == "13835403","category"] = "Accessory"          # Casing
products.loc[products.type == "5,74E+15","category"] = "Accessory"          # iMac
products.loc[products.type == "1364","category"] = "Network/Memory"         # Memory
products.loc[products.type == "12585395","category"] = "Electronic Accessory"# USB Adapter
products.loc[products.type == "1296","category"] = "Monitor"                # Monitor
products.loc[products.type == "1325","category"] = "Electronic Accessory"   # Cable
products.loc[products.type == "5384","category"] = "Multimedia"             # Headphones
products.loc[products.type == "1433","category"] = "Network/Memory"         # SSD
products.loc[products.type == "12215397","category"] = "Network/Memory"     # SSD
products.loc[products.type == "5398","category"] = "Multimedia"             # Speaker
products.loc[products.type == "1,02E+12","category"] = "Computer/Laptop"    # MacBook Pro
products.loc[products.type == "57445397","category"] = "Network/Memory"     # Memory
products.loc[products.type == "1,44E+11","category"] = "Service/Repair"     # Service/Repair
products.loc[products.type == "1334","category"] = "Network/Memory"         # Network/Memory
products.loc[products.type == "2158","category"] = ""        # MacBook Pro
products.loc[products.type == "2449","category"] = "Accessory"              # Apple watch Accessories
products.loc[products.type == "12655397","category"] = "Network/Memory"     # NAS Hard Drive
products.loc[products.type == "1229","category"] = "Electronic Accessory"   # Apple Pen Pointer
products.loc[products.type == "12995397","category"] = "Network/Memory"     # "Hard Drive ao Accessories" 
products.loc[products.type == "1515","category"] = "Electronic Accessory"   # Battery
products.loc[products.type == "13615399","category"] = "Electronic Accessory"# Charger 
products.loc[products.type == "13555403","category"] = "Accessory"          #Screen Protection
products.loc[products.type == "1405","category"] = "Tablet"                 # Graphic Tablet 
products.loc[products.type == "1230","category"] = "Electronic Accessory"   # Lightning Cable
products.loc[products.type == "118692158","category"] = "Computer/Laptop"   # iMac
products.loc[products.type == "1216","category"] = "Accessory"              # Apple Stand Mounts
products.loc[products.type == "24885185","category"] = "Wearable"           # Apple Watch
products.loc[products.type == "24895185","category"] = "Wearable"           # Apple Watch
products.loc[products.type == "21485407","category"] = "Service/Repair"     # iPhone Repair
products.loc[products.type == "1392","category"] = "Other"                  # backpacks
products.loc[products.type == "11821715","category"] = "Multimedia"         # iPod
products.loc[products.type == "8696","category"] = "Accessory"              # Apple Stand Mounts
products.loc[products.type == "9094","category"] = "Camera/Drone"           # Camera
products.loc[products.type == "13835403","category"] = "Accessory"          # Casing
products.loc[products.type.isin(["85641716","24811716","24821716","113291716",
                                 "113281716","113271716","21561716", "85651716",
                                 "51601716"]),"category"] = "Smartphone"    # iPhones
products.loc[products.type == "1231","category"] = "Service/Repair"         # Apple Protection/ Warrenty
products.loc[products.type == "1387","category"] = "Electronic Accessory"   # Mouse
products.loc[products.type == "12755395","category"] = "Network/Memory"     # HDD and Kits
products.loc[products.type == "13855401","category"] = "Electronic Accessory"# Keyboards
products.loc[products.type == "","category"] = ""

In [ ]:
products.category.value_counts()

like 80 % has been assigned. for now, thats okay. 
go on and check if other categories fall into mind or need to be found for top sales products

In [ ]:
products.loc[((products.category == "") | (products.category == "Mix")) & (products.name.str.contains('|'.join(["case"]), case=False)), "category"] = "Accessory" # casing
products.loc[((products.category == "") | (products.category == "Mix")) & (products.name.str.contains('|'.join(["charger"]), case=False)), "category"] = "Electronic Accessory" # Charger
products.loc[((products.category == "") | (products.category == "Mix")) & (products.name.str.contains('|'.join(["iMAC"]), case=False)) & (products.price > 500), "category"] = "Computer/Laptop" # iMac
products.loc[((products.category == "") | (products.category == "Mix")) & (products.name.str.contains('|'.join(["ipad"]), case=False)) & (products.brand == "Apple") & (products.price > 200), "category"] = "Tablet" # iPad
products.loc[((products.category == "") | (products.category == "Mix")) & (products.name.str.contains('|'.join(["Mac"]), case=False)) & (products.price > 400),"category"] = "Computer/Laptop" #Mac
products.loc[((products.category == "") | (products.category == "Mix")) & (products.name.str.contains('|'.join(["drone"]), case=False)), "category"] = "Camera/Drone" # Drone
products.loc[products.name.str.contains('|'.join(["used","like new", "refurbish","second hand","nearly new", "secondhand"]), case=False), "category"] = "Second Hand"

products.loc[(products.category == "") | (products.category == "Mix")].nlargest(30,"price")
products.loc[((products.category == "") | (products.category == "Mix")) & (products.name.str.contains(''.join(["nas"]), case=False))]

(pd.DataFrame(Counter(" ".join(products.loc[(products.category == "") | (products.category == "Mix"),"name"]
                            .replace(r'[^\w\s]|_', '', regex=True)
                            .replace(r' +', ' ',regex=True))
                            .split())
                .most_common(30))
    )

In [ ]:
products.category.value_counts()
products.sample(30)
products.loc[products.sku == "APP1970"]

cat_info("113291716")

Category for Second Hand Material
Category for Rest <- about 20%. That is not satisfactory, but as this process is really time-consuming, its good for now

In [ ]:
# Assign "miscellaneous" to the rest (~1000 products) - further categorize those later, if there is time
products.loc[products.category == "", "category"] = "Miscellaneous"

# Second Hand
products.loc[products.name.str.contains('|'.join(["used","like new", "refurbish","second hand","nearly new", "secondhand"]), case=False), "category"] = "Second Hand"

## Category approach using PandasAI

In [ ]:
products_ai = products.copy().drop(columns=["type", "in_stock", "discount_est", "promo_price_est", "sku"])
llm = OpenAI(api_token="API KEY HERE",
             model_name = 'gpt-3.5-turbo')

orders_smart = pai.SmartDataframe(products_ai, config={"llm": llm}, name="products", description="List of offered products")
Agent = pai.Agent(products_ai)

orders_smart.chat("what are the most expensive products?")
Agent.chat("what are the most expensive products?")
Agent.clarification_questions("what are the most expensive products?")
Agent.explain()

#Agent.chat("""categorize all products within a new column 'category_ai', based on the keywords within  "name" and "des" columns. Use the following categories (examples in parantheses) and adapt and/or expand if necessary. 
#              Categories: Computer/Laptop (MacBook, iMac), Tablet (iPad, graphic Tablet), Smartphone (iPhone), Wearable (Apple Watch), Network/Memory (external / USB Hard Drive, NAS Server, SSD),
#              Monitor, Electronic Asseccory (Cables, Battery, Charger, Lighning, Pen, USB Adapter, Mouse, Keyboard), Accessory (Cases, Stands, Mounts, Screen Protection), Multimedia (Headphones, Speaker, iPod),
#              Service (Repair, Parts, Protection Plan), Camera/Drone.
#           """)

#Agent.chat("""Based on the "name" and "description" columns, categorize each product into the correct category and create a new column 'category_ai' for the result. 
#The categories include Computer/Laptop, Tablet, Smartphone, Wearable, Network/Memory, Monitor, Electronic Accessory, Accessory, Multimedia, Service, and Camera/Drone. 
#Please adapt or expand the categories as necessary based on product keywords.""")

#Agent.chat("""Categorize all products into categories within a new column 'category_ai'. Base your assignment on the product name. The categories include Computer/Laptop, Tablet, Smartphone, Wearable, Network/Memory, Monitor, Electronic Accessory, Accessory, Multimedia, Service, and Camera/Drone.""")

#Agent.chat("Do not use 'other', but one of the existing category names from column 'category'. based on the product descript in 'name' and des'")


orders_smart.chat("""Given the following categories, write a script to categorize these products, using information
in the name and desc columns. Save the result in a new column "category_ai.
Categories: Computer/Laptop, Tablet, Smartphone, Wearable, Network/Memory, Monitor, Electronic Accessory, Accessory, Multimedia, Service, and Camera/Drone
#
# - Computer/Laptop: 'Lenovo ThinkPad X1 Carbon', 'Apple MacBook Pro', 'iMac'
# - Tablet: 'Apple iPad Air', 'Samsung Galaxy Tab S7'
# - Smartphone: 'Samsung Galaxy S21', 'Apple iPhone 12'
# - Wearable: 'Apple Watch Series 6', 'Fitbit Versa 3'
# - Network/Memory: 'Synology DS218j NAS', 'Samsung EVO 1TB SSD'
# - Monitor: 'Dell Ultrasharp 24 Monitor', 'ASUS ProArt Display PA278QV'
# - Electronic Accessory: 'Anker PowerCore Portable Charger', 'Belkin Boost Up Wireless Charging Pad'
# - Accessory: 'OtterBox Defender Series Phone Case', 'JETech Screen Protector'
# - Multimedia: 'Sony WH-1000XM4 Wireless Headphones', 'Bose SoundLink Mini Bluetooth Speaker'
# - Service/Repair: 'AppleCare Protection Plan', 'Samsung Premium Care'
# - Camera/Drone: 'Canon EOS R5 Mirrorless Camera', 'DJI Mavic Air 2 Drone'

# Consider the main function of the product as described by its name for categorization.
""")
print(orders_smart.last_code_generated)
products_ai.sample(30)
products.nlargest(5, "price")

##  Save Data

In [ ]:
orders.info()
items.info()
products.info()

# save to csv
items.to_csv(folder+'items_clean.csv', index=False)
products.to_csv(folder+'products_clean.csv', index=False)
orders.to_csv(folder+'orders_clean.csv', index=False)
brands.to_csv(folder+'brands_clean.csv', index=False)